<a href="https://colab.research.google.com/github/ChaeJoowon/KookminUv_Joowon/blob/master/%5BHW01%5D_AutoEncoderLightning_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📝 과제 제출 정보  

| 📌 항목  | ✏️ 입력 |
|---------|-------|
| **학과** | `소프트웨어학과` |
| **학번** | `20203149` |
| **이름** | `채주원` |

🚨 위 과제 제출 정보를 정확히 기입하지 않으면 감점됩니다.

---

## [HW01] AutoEncoderLightning


`[HW01] Autoencoder`에서 진행한 내용을 [PyTorch Lightning](https://lightning.ai/docs/pytorch/stable/)을 이용해 재구현하도록 한다.

- PyTorch Lightning은 반복적인 코드를 줄이고, 더 깔끔하고 구조화된 방식으로 딥러닝 모델을 개발할 수 있도록 도와주는 프레임워크이다.
- 사용 방법은 [여기](https://lightning.ai/docs/pytorch/stable/starter/introduction.html)를 참고하도록 하자.

---

# PyTorch Lightning 설치

In [ ]:
!pip install lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 960.9/960.9 kB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 필요한 모듈 임포트

In [ ]:
import sys
import time
import numpy as np
import matplotlib.pyplot as plt

import torch                                      # 파이토치 임포트
import torch.nn as nn                             # nn 모듈 임포트
import torch.nn.functional as F

import torchvision                                # torchvision 임포트
import torchvision.utils as utils
import torchvision.transforms as transforms       # numpy 이미지에서 tensor 이미지로 변경하기 위한 모듈
import torchvision.datasets as datasets           # pytorch에 내장된 dataset을 불러오기 위한 모듈

import lightning as L                             # PyTorch Ligthning 모듈 임포트

from tqdm import tqdm

print("Python:", sys.version)
print("Numpy:", np.__version__)

print("Torch: ", torch.__version__)               # 파이토치 버전을 확인하도록 한다.
print("Torchvision: ", torchvision.__version__)   # 토치비전 버전을 확인하도록 한다.

Python: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
Numpy: 2.0.2
Torch:  2.6.0+cu124
Torchvision:  0.21.0+cu124


## Hyperparameter 설정
* batch size, learning rate, epoch을 설정
* 잠재 벡터(latent vector) $z$의 크기 설정

In [ ]:
batch_size    = 100
learning_rate = 0.001
num_epochs    = 10
n_dim = 2

# 데이터셋

In [ ]:
import torchvision.transforms as transforms
from torchvision.datasets import MNIST

# download path 정의
download_root = './MNIST_DATASET'


# PIL image (H x W x C) -> torch tensor (C x H x W) and
# [0, 255] -> [0.0, 1.0]
mnist_transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = MNIST(download_root,
                      transform=mnist_transform,
                      train=True,
                      download=True)

test_dataset = MNIST(download_root,
                     transform=mnist_transform,
                     train=False,
                     download=False)

100%|██████████| 9.91M/9.91M [00:00<00:00, 16.3MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 476kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.44MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.94MB/s]


## MNIST DataLoader 정의

[파이토치 데이터로더](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)는 데이터셋을 배치단위로 묶어 순회(iteration)할 수 있도록 한다.

아래 코드는 데이터셋에 대한 데이터로더를 만들고, 각 배치가 어떤 텐서 형태로 구성되어 있는지 확인하는 코드이다.
* 데이터로더에서는 이미지들이 배치단위로 묶였기 때문에, B x C x H x W 모양(shape)의 텐서이며, 데이터타입(dtype)은 `torch.float32`가 되었음을 확인하자.
* 데이터로더에서는 레이블들이 배치단위로 묶였기 때문에, B 모양(shape)의 텐서이며, 데이터타입(dtype)은 `torch.int64`가 되었음을 확인하자.

train_dataset과 test_dataset은 동일 형태이므로, train_dataset의 데이터로더에 대해서만 확인하였다.


In [ ]:
from torch.utils.data import DataLoader

# dataloader 정의
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset,
                         batch_size=batch_size,
                         shuffle=False)

## 네트워크 모델 설계
* 인코더 모델: `torch.nn.Sequential` 모듈로 모델로 설계
  + InputLayer로 (1,28,28) 영상을 받고, Linear 레이어를 여럿 통과시킨 후, 출력으로 n_dim차원 벡터가 나오도록 함.
* 디코더 모델: `torch.nn.Sequential` 모듈로 모델로 설계
  + InputLayer에서 n_dim차원 벡터를 받고, Linear 레이어를 여럿 통과시킨 후, 출력으로 (1,28,28) 영상이 나오도록 함.
* 오토인코더 모델: 인코더, 디코더를 결합하여 설계

### 인코더 모델 정의
* (1, 28, 28) 영상을 입력으로 받도록 입력 레이어 정의
* Flatten으로 입력 텐서를 784-vector로 벡터라이즈
* Fully connected layer로 784 > 256 > 128 > 32 > n_dim 로 차원 축소

### 디코더 모델 정의
* Fully connected layey로 n_dim > 32 > 128 > 256 > 784로 차원 확대
* 784-vector를 Reshape을 통해 (28, 28)의 텐서로 변환

In [ ]:
enc = nn.Sequential(
  nn.Flatten(),    # 784 = 1 x 28 x 28
  nn.Linear(784, 256),
  nn.ReLU(),
  nn.Linear(256, 128),
  nn.ReLU(),
  nn.Linear(128, 32),
  nn.ReLU(),
  nn.Linear(32, n_dim)
)

dec = nn.Sequential(
    nn.Linear(n_dim, 32),
    nn.ReLU(),
    nn.Linear(32, 256),
    nn.ReLU(),
    nn.Linear(256, 784),
    nn.Unflatten(1, torch.Size([1, 28, 28])),
    nn.Sigmoid(),
)

# PyTorch Lightning을 사용한 모델 정의

- `LightningModule`을 서브클래싱한 AutoEncoder 클래스를 정의하자.
-  `AutoEncoder` 클래스 내부에 학습 루프, 최적화 설정 등을 모두 캡슐화하여 구현



In [ ]:
# TODO

class AutoEncoder(L.LightningModule):
  def __init__(self, enc, dec):
    super().__init__()
    self.enc = enc
    self.dec = dec

  def training_step(self, batch, batch_idx):
    ## training step은 훈련 단계의 각 step에서 진행하는 일을 정의함.
    pass

  def configure_optimizers(self):
    pass

  def forward(self, x):
    pass



# 모델 인스턴스 정의
- `AutoEncoder` 클래스의 인스턴스 `ae`를 정의한다.
- 모델 인스턴스 `ae`가 어떤 디바이스에 있는지 조사한다.

In [ ]:
# init the autoencoder
ae = AutoEncoder(enc, dec)

print(ae.device)

cpu


## 네트워크 모델 구조 확인
딥러닝 모델의 입출력 구조를 다음과 같이 확인한다.  여기서 중요한 사항은 다음을 확인하는 것이다.

1. 레이어의 연결구조,  
2. 레이어별 입출력 텐서의 차원,
3. 학습할 파라메터 개수

* Lightning이 모델을 어떤 디바이스에 보유하고 있는지 `device=str(ae.device)`로 확인하면서 사용해야 함.

In [ ]:
!pip install torchsummary

In [ ]:
from torchsummary import summary

summary(ae, (1, 28, 28), device=str(ae.device))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #


AttributeError: 'int' object has no attribute 'numpy'

Lightning 모듈에서 지원하는 summarize() 함수로 모델의 구조를 텍스트로 프린트할 수 있음.
* 모델이 어떤 디바이스에 있는지 신경쓸 필요 없음.
* 하지만 torchsummary 보다 간단한 방식으로만 확인 가능함.

In [ ]:
from pytorch_lightning.utilities.model_summary import summarize

print(summarize(ae, max_depth=2))  # 요약 출력

## 훈련 전, 네트워크 모델을 함수로서 활용
* AutoEncoder ae를 모델로 구성했기 때문에, 지금부터 함수로서 활용 가능
  + 단, ae 함수는 batch 단위로 수행됨을 명심할 것.  
    - 단순히, (1, 28, 28) -> ae -> (1, 28, 28)로 동작하지 않고,
    - batch 단위로 (?, 1, 28, 28) -> ae -> (?, 1, 28, 28)로 병렬처리됨.
* 지금은 훈련 전 네트웍이기 때문에 함수로서는 작동하지만 노이즈 출력만 나올 뿐, 정상적인 출력이 나오는 함수로 동작하지 않음.

In [ ]:
# TODO


## 딥러닝 모델 학습

Lightning의 `Trainer`를 만들고, `fit()` 함수를 통해 딥러닝 모델을 학습시킴.

In [ ]:
# TODO


### 트레이닝셋을 이용해 학습 후 결과 시각화
학습이 끝난 autoencoder에 대해 train_dataset에 대한 결과 시각화

In [ ]:
# TODO


# 인코더 / 디코더 모델을 각각 따로 함수로서 활용하기
오토인코더 네트웍 전체가 아닌 `enc()` 부분과 `dec()` 부분을 각각 수행할 수 있다.
* 특정 예제에 대한 인코딩 결과와 디코딩 결과를 따로 확인한다.

In [ ]:
# TODO


인코딩 결과와 유사한 좌표값을 디코딩에 보내도 유사한 결과가 나옴을 확인

In [ ]:
# TODO


## TODO
1. encoder를 거쳐 나온 representation z를 GT label 별로 다른색을 주어 z의 분포를 가시화한다.
2. z의 분포를 보고 encoder가 label 별로 discriminative한 representation을 만들어내는지 여부를 확인한다.
3. 만일 discriminative하지 않다면, autoencoder가 discriminative한 representation z를 학습하도록 z의 차원을 바꿔가면서 실험을 진행한다.
4. 결과에 대한 분석을 작성한다.

## 인코딩 결과 가시화
오토인코더의 encoder가 만들어 내는 representation인 z 값을 가시화 한다.

In [ ]:
# TODO


## TODO: 디코더를 이용한 Generative Model 구성
1. z 공간의 임의의 위치를 sampling한 후, 이를 decoder의 입력으로 주는 방식으로 generative model을 구축한다.
2. 결과로 만들어지는 이미지는 어떤 특징을 가지고 있는지 분석한다.

In [ ]:
## TODO


결과 가시화

In [ ]:
## TODO
